In [1]:
from autogenerate import autogenerate

In [2]:
# The index convenction is arbitrary, 
# orientation is not important here

vertices = [{0},{1},{2}]

edges = [
        {1,2},
        {2,0},
        {0,1},
]

edges_oriented = [
        [1,2],
        [2,0],
        [0,1],    
]

s = [vertices,edges]
len(s[0])


3

In [3]:
# Generates all possible tuples, note that some might not be valid
def all_tuples(s):
    return [[i,j] for i in range(len(s[0])) for j in range(len(s[1]))]

# Check if a tuple is valid, i.e. if the simplex of dimension d-1 is a face of a simplex of dimension d
def valid_tuple(t,s):
    return s[0][t[0]].issubset(s[1][t[1]]) 

# Check if a tuple is ccw
def ccw_tuple(t,s):
    assert(valid_tuple(t,s))
    assert (edges_oriented[t[1]][0] in s[0][t[0]]) or (edges_oriented[t[1]][1] in s[0][t[0]])
    return edges_oriented[t[1]][0] in s[0][t[0]]

# valid = [t for t in all_tuples(s) if valid_tuple(t,s)]



In [4]:
# Enumerates all valid tuples similar to t, but with a different value in the slot d
# There must be 2 of them, and this function returns the one that is different than t
def switch(t,d,s):
    candidates = []
    for i in range(len(s[d])):
        t2 = t.copy()
        t2[d] = i
        candidates.append(t2)
    valid = [x for x in candidates if valid_tuple(x,s)]
    assert(len(valid) == 2)
    valid.remove(t)
    assert(len(valid) == 1)
    return valid[0]

In [5]:
def list_to_array(list_of_lists,name):
    # Get the number of rows and columns in the list of lists
    rows = len(list_of_lists)
    cols = len(list_of_lists[0])

    # Create the C++ code for the static array of arrays
    cplusplus_code = "static long " + name +"[" + str(rows) + "][" + str(cols) + "] = {\n"

    # Loop through each row in the list of lists
    for row in list_of_lists:
        # Add the opening brace for the row
        cplusplus_code += "  {"

        # Loop through each element in the row
        for element in row:
            # Add the element to the row
            cplusplus_code += str(element) + ", "

        # Remove the trailing comma and space from the row
        cplusplus_code = cplusplus_code[:-2]

        # Add the closing brace for the row
        cplusplus_code += "},\n"

    # Remove the trailing comma and newline from the C++ code
    cplusplus_code = cplusplus_code[:-2] + "\n};\n"

    return cplusplus_code

In [6]:
# Builds a table for the switch operation of dimension d
def table(d,s):
    # Generates a table for switch_vertex
    sv = all_tuples(s)

    for i in range(len(sv)):
        if not valid_tuple(sv[i],s):
            sv[i] = [-1,-1]
        else:
            sv[i] = switch(sv[i],d,s)

    return sv

In [7]:
# Builds a table for the switch operation of dimension d
def table_ccw(s):
    # Generates a table for switch_vertex
    sv = all_tuples(s)
    out = [-1 for i in sv]

    for i in range(len(sv)):
        if not valid_tuple(sv[i],s):
            out[i] = -1
        else:
            if ccw_tuple(sv[i],s):
                out[i] = 1
            else:
                out[i] = 0
            
    return out

# table_ccw("auto_2d_table_ccw",s)

In [8]:
def table_complete_tuple(d,s):
    # Generates a table for switch_vertex
    sv = [t for t in all_tuples(s) if (valid_tuple(t,s) and ccw_tuple(t,s))]
    out = []
    for c in range(len(s[d])):
        for i in range(len(sv)):
            if c == sv[i][d]:
                out.append(sv[i])
                break
    
    assert(len(out) == len(s[d]))
            
    return out

#table_complete_tuple("complete_vertex",0,s)


In [9]:
def list_pair_to_array(lists, name, comment = None):
    r = {"name": name, "data": lists}
    if comment is not None: r["comment"] = comment
    return r

In [10]:
tri_data = {"namespace": "tri_mesh", "tables": [
list_pair_to_array(edges,"auto_2d_edges", "lvids for a particular leid"),
list_pair_to_array(table(0,s),"auto_2d_table_vertex", "vertex switch's (tuple_index -> [lvid,leid])"),
list_pair_to_array(table(1,s),"auto_2d_table_edge", "edge switch's (tuple_index -> [lvid,leid])"),
list_pair_to_array(table_ccw(s),"auto_2d_table_ccw", "(tuple_index) -> is_ccw"),
list_pair_to_array(table_complete_tuple(0,s),"auto_2d_table_complete_vertex", "lvid -> a ccw [lvid,leid]"),
list_pair_to_array(table_complete_tuple(1,s),"auto_2d_table_complete_edge", "leid -> a ccw [lvid,leid]")]}


In [14]:
autogenerate(["autogenerated_data"], tri_data)